# Homework 3

**Please answer all of the bolded questions and include your plots in a pdf document submitted along with this notebook and your homework solutions.**

For the last homework assignment, we looked a binary system of particles mixing within our simulation box. For this homework, we are going to look at movement of particles throughout the box, and determine ways to quantify this diffusion. As always, we will begin by importing all of our needed modules.

In [ ]:
import numpy as np 
import hoomd
import gsd.hoomd
import matplotlib.pyplot as plt

Now, we need to set up the simulation. This is going to be the same process as all of our previous simulations, so I'll let you do it! Create the device and simulation object for this simulation. 

In [ ]:
#device
#sim

Now that we have created our simulation object, we can load in our initial configuration. We're going to do it a little bit differently this time, but we will get the same result as usual. 

In [ ]:
with gsd.hoomd.open('binary_config.gsd', 'r') as f:
    sim.create_state_from_snapshot(f[0])
f.close()

Now that you've loaded in the initial configuration, we can set up the integrator and forces in the system.  Before do this, however, I'd like you to visualize the initial configuration of the system using OVITO. In this system, you have 1000 particles, 800 type A particles and 200 type B particles. One of the key differences between these particles is their size. Type B's diameter is $0.5\sigma$, while type A has a diameter of $1\sigma$. This means that the type A particles are twice the size of the type B particles. We will be using the Lennard-Jones potential as usual. **Fill in the sigma parameters for the LJ force object.**

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)
sim.operations.integrator = integrator


cell = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=cell, default_r_cut=2**(1/6))

lj.params[('typeA', 'typeA')] = dict(epsilon=1, sigma=)
lj.params[('typeB', 'typeB')] = dict(epsilon=1, sigma=)
lj.params[('typeA', 'typeB')] = dict(epsilon=1, sigma=)
integrator.forces.append(lj)

nvt = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All(), 
                                      thermostat=hoomd.md.methods.thermostats.MTTK(kT=1.0, 
                                                                tau=sim.operations.integrator.dt*100))
integrator.methods.append(nvt)

sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1.0)

thermo_properties = hoomd.md.compute.ThermodynamicQuantities(filter=hoomd.filter.All())
sim.operations.computes.append(thermo_properties)

To make our analysis easier, we are going to create 3 separate loggers. We will have one main logger that logs the data for the entire system, one that logs data for just the A particles, and one that logs data for just the B particles. Make sure that you make each filename unique so that you don't accidentally overwrite your log files. 

In [ ]:
logger = hoomd.logging.Logger()

typeA_writer = hoomd.write.GSD(filename='trajA.gsd', trigger=hoomd.trigger.Periodic(100), 
                              mode='wb', filter=hoomd.filter.Type(['typeA']), 
                              dynamic=['property', 'particles/image'], logger=logger)
typeA_writer.write_diameter=True
sim.operations.writers.append(typeA_writer)

typeB_writer = hoomd.write.GSD(filename='trajB.gsd', trigger=hoomd.trigger.Periodic(100), 
                              mode='wb', filter=hoomd.filter.Type(['typeB']), 
                              dynamic=['property', 'particles/image'], logger=logger)
typeB_writer.write_diameter=True
sim.operations.writers.append(typeB_writer)


logger.add(thermo_properties)
gsd_writer = hoomd.write.GSD(filename='log.gsd', trigger = hoomd.trigger.Periodic(1000), 
                             mode='wb', filter=hoomd.filter.All(), 
                             dynamic=['property', 'particles/image'], logger=logger)
gsd_writer.write_diameter=True
sim.operations.writers.append(gsd_writer)



table_logger = hoomd.logging.Logger(categories=['scalar', 'string'])
table_logger.add(sim, quantities=['timestep', 'tps', 'walltime'])
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=100000),
                          logger=table_logger)
sim.operations.writers.append(table)

Now, we can run the simulation! We want to run this simulation for a long time to ensure we get accurate data. 

In [ ]:
sim.run(1000000)
gsd_writer.flush()
typeA_writer.flush()
typeB_writer.flush()

# Analysis

To quantify the diffusion happening in our system, we are going to calculated the Mean-Squared Displacement (MSD) of the particles in our system. The MSD measures how far a particle has travelled from its initial position over time. There are many ways to calculate the MSD, but for this assignment we will be calculating it directly using the equation $${MSD}(t) = \frac{1}{N_{particles}} \sum_{i=1}^{N_{particles}} (r_i(t)-r_i(0))^2 $$

We can calculate the MSD using freud.

In [ ]:
import freud

First, we need to load our trajectory files and extract the relevant data from our simulation. To calculate the MSD with freud, we need the box dimensions, the particle positions, and the particle image flags. The image flags tell us how many times a particle crosses a periodic boundary condition. Using these image flags, freud will calculate the 'unwrapped' particle positions, aka the true distance that the particles have travelled. We want to compare the MSDs of our A and B particles, so we need to load each trajectory separately. 

In [ ]:
trajA = gsd.hoomd.open('trajA.gsd', mode='r')
trajB = gsd.hoomd.open('trajB.gsd', mode='r')

NA = trajA[0].particles.N
NB = trajB[0].particles.N
nframes = trajA.__len__()
imgA = np.zeros([nframes, NA, 3])
imgB = np.zeros([nframes, NB, 3])
posA = np.zeros([nframes, NA, 3])
posB = np.zeros([nframes, NB, 3])
for i in range(nframes):
    imgA[i] = trajA[i].particles.image
    posA[i] = trajA[i].particles.position    
    imgB[i] = trajB[i].particles.image
    posB[i] = trajB[i].particles.position

Now that we've gotten gotten our particle data, we can compute the MSD. For more information about freud's MSD module, take a look at their [documentation](https://freud.readthedocs.io/en/latest/modules/msd.html). 

In [ ]:
box = freud.box.Box.from_box(trajA[0].configuration.box)
msdA = freud.msd.MSD(box, 'direct').compute(posA, imgA)
msdB = freud.msd.MSD(box, 'direct').compute(posB, imgB)

Now, let's plot the MSD! First, we're going to look at a log-log plot of the MSD vs. time. If you look at the slopes of the lines on the plot, you'll notice two distinct regions, or diffusion regimes. **What are the log-log slopes of these two regions, and how do they differ from each other? What do you think causes the two diffusion regimes?**

In [ ]:
timesteps = np.linspace(0, 1000000, nframes)
fig, ax = plt.subplots()

ax.loglog(timesteps, msdA.msd, label='typeA')
ax.loglog(timesteps, msdB.msd, label='typeB')
ax.legend()
ax.set_xlabel('timesteps')
ax.set_ylabel('MSD')
plt.show()

Now that we've looked at the log-log plots of our MSD, let's look at our normal plot. We can use this to calculate our diffusion coefficient. **Plot the MSD vs. time for the two particle types.**

In [ ]:
#Plot the MSD vs. time for the two particle types

The MSD is related to the diffusion coefficient for simple systems by the Einstein expression $$ D = \lim_{t\to\inf}\frac{1}{6N_{particles}t}\left\langle\sum_{i=1}^{N_{particles}}\left[r_i(t)-r_i(0)\right]^2\right\rangle $$

Using this expression, we can calculate the diffusion coefficient of our system by fitting the tail end of the MSD (aka the MSD as $t\to\inf$) to a linear equation and finding the slope of that line. The diffusion coefficient can then be estimated as $$D=\frac{1}{6}\frac{d(MSD)}{dt}$$ 

**Calculate the diffusion coefficient for each particle type and discuss why the particles behave differently in this simulation.**  